# Image Classification Model Using LeNet

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm

In [2]:
class HarDataset(Dataset):
    def __init__(self, root_dir, data, transform=None):
        self.root_dir = root_dir
        self.data_df = data
        self.transform = transform
        self.label_map = self._create_label_map()

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_df.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.data_df.iloc[idx, 1]
        label = self.label_map[label]

        if self.transform:
            image = self.transform(image)

        return image, label

    def _create_label_map(self):
        unique_labels = sorted(self.data_df['label'].unique())
        label_map = {label: idx for idx, label in enumerate(unique_labels)}
        return label_map

In [3]:
# Define transforms for data augmentation

# Use this for colored images
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

# Use this for BnW images
# transform = transforms.Compose([
#     transforms.Resize((32, 32)),
#     transforms.Grayscale(num_output_channels=1),  # Convert to black and white
#     transforms.ToTensor(),
# ])

In [4]:

# Define the paths 

path = r'/Users/azeez/Desktop/AI/Personal Projects/GitHub/Computer-Vision-using-PyTorch/Human Action Recognition' 
folder = 'train' 
activity = 'cycling' 
img_name = 'Image_93.jpg' 

# construct the full image path 
img_path = os.path.join(path, folder, activity, img_name)

# Open and display the image 
image = Image.open(img_path)
image.show() 


In [5]:
print(image.size)

(385, 131)


In [6]:

print(image.mode)

RGB


In [ ]:
train_root_dir = r'/Users/azeez/Desktop/AI/Personal Projects/GitHub/Computer-Vision-using-PyTorch/Human Action Recognition/train'
train_file = r'C:\Users\Deepak\Downloads\Computer Vision Resources\Human Action Recognition\Modified_Training_set.csv'
test_root_dir = r'/Users/azeez/Desktop/AI/Personal Projects/GitHub/Computer-Vision-using-PyTorch/Human Action Recognition/test'
test_file = r'C:\Users\Deepak\Downloads\Computer Vision Resources\Human Action Recognition\Reduced_Testing_set.csv'